### After several days of getting my scraper to a reasonable level, now it's time to build a simple model that describes the data 

### Current
since I lack time-series data for each location this will simply use know information as well as maybe some exogeneous information to predict prices.

- determine what predictors are releavent on a national level to estimate prices
- determine what predictors are releavent on a local level to restimate prices

### FUTURE
forecast prices

- locate additional sources of information to be used for backtests
- seasonality model?



In [1]:
### import the relevant libraries ###
import pandas as pd
import numpy as np
from datetime import datetime as dt
pd.options.display.float_format = "{:.0f}".format

import re

#### sklearn ###
from sklearn.preprocessing import StandardScaler
retail_scaler = StandardScaler()

# for making a Custom Scaler
from sklearn.base import BaseEstimator, TransformerMixin

# import the LogReg model from sklearn
from sklearn.linear_model import LogisticRegression

# import the 'metrics' module, which includes important metrics we may want to use
from sklearn import metrics

In [2]:
# create the Custom Scaler class

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    # init or what information we need to declare a CustomScaler object
    # and what is calculated/declared as we do
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        
        # scaler is nothing but a Standard Scaler object
        self.scaler = StandardScaler(copy,with_mean,with_std)
        # with some columns 'twist'
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
    # the fit method, which, again based on StandardScale
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    # the transform method which does the actual scaling

    def transform(self, X, y=None, copy=None):
        
        # record the initial order of the columns
        init_col_order = X.columns
        
        # scale all features that you chose when creating the instance of the class
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        
        # declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        # return a data frame which contains all scaled features and all 'not scaled' features
        # use the original order (that you recorded in the beginning)
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [3]:
unfiltered_data = pd.read_csv("DATA/scrape_files/Master.csv")
unfiltered_data.shape


(34959, 15)

In [4]:
unfiltered_data[unfiltered_data.sqft.str.contains("-") | unfiltered_data.sqft.str.contains("NaN")].count()


Unnamed: 0     2147
name           2147
address        2147
unit           2147
sqft           2147
bed            2147
bath           2147
price          2147
city           2147
state          2147
zipcode        2147
description    2147
details        2147
url            2147
date           2147
dtype: int64

In [5]:
unfiltered_data[~unfiltered_data.sqft.str.contains("-", na=False)].count()

Unnamed: 0     32812
name           32812
address        32812
unit           32070
sqft           32066
bed            32812
bath           32812
price          32812
city           32812
state          32812
zipcode        32812
description    32812
details        32812
url            32812
date           32812
dtype: int64

In [6]:
# remove the 2k+ appartment summary listsings, which (I believe moves the data where the column was shifted)
uf_data = unfiltered_data[~unfiltered_data.sqft.str.contains("-", na=False)]
uf_data = uf_data[~uf_data.unit.str.contains("Bed", na=False)]
uf_data = uf_data[uf_data.unit != 'home'] # get rid of homes/condos for rent (for now)

In [7]:
uf_data.sqft.replace(regex=True, to_replace=r'\D', value='', inplace=True)
uf_data.sqft.fillna(0, inplace=True)

uf_data.unit.replace(regex=True, to_replace=r'\D', value='', inplace=True)
uf_data.bed.replace(regex=True, to_replace=r'\D', value='', inplace=True)
uf_data.bath.replace(regex=True, to_replace=r'\D', value='', inplace=True)
uf_data.price.replace(regex=True, to_replace=r'\D', value='', inplace=True)

In [8]:
uf_data.price.replace(regex=True, to_replace=r'\D', value='', inplace=True)

In [9]:
def detail_string_to_dict(detail):
    """ something """
    my_dict = {}
    for d in detail.split(','):
        if ":" in d:
            it = iter([(d.split(":")[0]).strip(), (d.split(":")[1]).strip()])
            mydict = dict(zip(it, it))
            my_dict.update(mydict)

        else:
            l = (d.strip()+":True").split(":")
            it = iter(l)
            myd = dict(zip(it, it))
            # print(myd)
            my_dict.update(myd)
    return my_dict

In [10]:
uf_data['details'] = uf_data.details.apply(detail_string_to_dict)

In [11]:
uf_data.head()

,Unnamed: 0,name,address,unit,sqft,bed,bath,price,city,state,zipcode,description,details,url,date
1,1,Avanti Apartments,10697 W Centennial Pkwy,2140,753,1,1,1127,Las Vegas,NV,89166,This apartment is located at 10697 W Centennia...,"{'Heating': 'Other', 'Days on Market': '41 Day...",https://www.trulia.com/c/nv/las-vegas/avanti-a...,11/17/2020
2,2,Avanti Apartments,10697 W Centennial Pkwy,1119,771,1,1,1124,Las Vegas,NV,89166,This apartment is located at 10697 W Centennia...,"{'Heating': 'Other', 'Days on Market': '41 Day...",https://www.trulia.com/c/nv/las-vegas/avanti-a...,11/17/2020
3,3,Avanti Apartments,10697 W Centennial Pkwy,1078,849,1,1,1335,Las Vegas,NV,89166,This apartment is located at 10697 W Centennia...,"{'Heating': 'Other', 'Days on Market': '41 Day...",https://www.trulia.com/c/nv/las-vegas/avanti-a...,11/17/2020
4,4,Avanti Apartments,10697 W Centennial Pkwy,1092,820,1,1,1347,Las Vegas,NV,89166,This apartment is located at 10697 W Centennia...,"{'Heating': 'Other', 'Days on Market': '41 Day...",https://www.trulia.com/c/nv/las-vegas/avanti-a...,11/17/2020
5,5,Avanti Apartments,10697 W Centennial Pkwy,3030,801,1,1,1123,Las Vegas,NV,89166,This apartment is located at 10697 W Centennia...,"{'Heating': 'Other', 'Days on Market': '41 Day...",https://www.trulia.com/c/nv/las-vegas/avanti-a...,11/17/2020


In [12]:
deets = uf_data.details
deets.head()

1    {'Heating': 'Other', 'Days on Market': '41 Day...
2    {'Heating': 'Other', 'Days on Market': '41 Day...
3    {'Heating': 'Other', 'Days on Market': '41 Day...
4    {'Heating': 'Other', 'Days on Market': '41 Day...
5    {'Heating': 'Other', 'Days on Market': '41 Day...
Name: details, dtype: object

In [13]:
my_list = []
for deet in deets:
    # print(deet)
    for d in deet:
        if d not in my_list:
            my_list.append(d)
        else:
            pass

    # my_list.append(list(deet.keys()))
my_list

['Heating',
 'Days on Market',
 'Year Built',
 'Property Type',
 'Microwave',
 'Air Conditioning',
 'Hot Tub or Spa',
 'Balcony',
 'Parking',
 'Pool',
 'Cats',
 'dogs allowed',
 'Fitness Center',
 'Additional Storage',
 'Dishwasher',
 'Disposal',
 'Dryer',
 'Laundry Facilities',
 'Washer',
 'Window Coverings',
 'Disabled Access',
 'Gated Entry',
 'Patio',
 'Assigned Parking Space',
 'Ceiling Fan',
 'Garage',
 'Refrigerator',
 'Barbeque Area',
 'Garden',
 'Sprinkler System',
 'Off Street',
 'Security System',
 'Views',
 'Fireplace',
 'Deck',
 'Sauna',
 'Electric',
 'HOA Fee',
 'Lawn',
 'Other',
 'Elevator',
 'Basketball Court',
 'Tennis Court',
 'Garage Detached',
 'Lot Size',
 'Furnished',
 'Gas',
 'Vaulted Ceiling',
 'Cats allowed',
 'Sport Court',
 'Skylight',
 'Double Paned Windows',
 'Intercom',
 'Handrails',
 'On Street',
 'Dogs allowed',
 'See Virtual Tour',
 'Propane Butane',
 'Wet Bar',
 'Dock',
 'Waterfront',
 'None',
 'Porch',
 'Pond',
 'Solar',
 'Doorman',
 'Forced Air',
 'R

In [14]:
# deets = uf_data.details
# for deet in deets[:10]:
#     # print(deet)
#     for d in deet:
#         if d in my_list:
#             uf_data[d] = deet[d]
#         else:
#             pass
# uf_data.head()

In [15]:
final = pd.DataFrame(deets.T.to_dict())
final = final.T
final.head()

,Heating,Days on Market,Year Built,Property Type,Microwave,Air Conditioning,Hot Tub or Spa,Balcony,Parking,Pool,...,Wet Bar,Dock,Waterfront,None,Porch,Pond,Solar,Doorman,Forced Air,Rv Parking
1,Other,41 Days on Trulia,2009,Apartment,True,True,True,True,None,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Other,41 Days on Trulia,2009,Apartment,True,True,True,True,None,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Other,41 Days on Trulia,2009,Apartment,True,True,True,True,None,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Other,41 Days on Trulia,2009,Apartment,True,True,True,True,None,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Other,41 Days on Trulia,2009,Apartment,True,True,True,True,None,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
final.replace("True", value=int(1), inplace=True)
final.replace("None", value=int(0), inplace=True)
final.replace("NaN", value=int(0), inplace=True)
final.replace("nan", value=int(0), inplace=True)
final


,Heating,Days on Market,Year Built,Property Type,Microwave,Air Conditioning,Hot Tub or Spa,Balcony,Parking,Pool,...,Wet Bar,Dock,Waterfront,None,Porch,Pond,Solar,Doorman,Forced Air,Rv Parking
1,Other,41 Days on Trulia,2009,Apartment,1,1,1,1,0,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,Other,41 Days on Trulia,2009,Apartment,1,1,1,1,0,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,Other,41 Days on Trulia,2009,Apartment,1,1,1,1,0,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,Other,41 Days on Trulia,2009,Apartment,1,1,1,1,0,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,Other,41 Days on Trulia,2009,Apartment,1,1,1,1,0,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34951,Forced Air,16 Days on Trulia,1972,Apartment,1,nan,nan,nan,0,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
34953,Forced Air,48 Days on Trulia,1963,Apartment,nan,nan,nan,nan,0,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
34954,Forced Air,48 Days on Trulia,1963,Apartment,nan,nan,nan,nan,0,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
34956,NaN,NaN,NaN,Apartment,nan,nan,nan,nan,0,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [17]:
final['Days on Market'].replace(regex=True, to_replace=r'\D', value='', inplace=True)
final

,Heating,Days on Market,Year Built,Property Type,Microwave,Air Conditioning,Hot Tub or Spa,Balcony,Parking,Pool,...,Wet Bar,Dock,Waterfront,None,Porch,Pond,Solar,Doorman,Forced Air,Rv Parking
1,Other,41,2009,Apartment,1,1,1,1,0,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,Other,41,2009,Apartment,1,1,1,1,0,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,Other,41,2009,Apartment,1,1,1,1,0,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,Other,41,2009,Apartment,1,1,1,1,0,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,Other,41,2009,Apartment,1,1,1,1,0,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34951,Forced Air,16,1972,Apartment,1,nan,nan,nan,0,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
34953,Forced Air,48,1963,Apartment,nan,nan,nan,nan,0,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
34954,Forced Air,48,1963,Apartment,nan,nan,nan,nan,0,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
34956,NaN,NaN,NaN,Apartment,nan,nan,nan,nan,0,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [18]:
# use get_dummies on a subset of columns?

In [19]:
deets = unfiltered_data.details

In [20]:
print(dir(deets))


['T', '_AXIS_ALIASES', '_AXIS_IALIASES', '_AXIS_LEN', '_AXIS_NAMES', '_AXIS_NUMBERS', '_AXIS_ORDERS', '_AXIS_REVERSED', '_HANDLED_TYPES', '__abs__', '__add__', '__and__', '__annotations__', '__array__', '__array_priority__', '__array_ufunc__', '__array_wrap__', '__bool__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__div__', '__divmod__', '__doc__', '__eq__', '__finalize__', '__float__', '__floordiv__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__iand__', '__ifloordiv__', '__imod__', '__imul__', '__init__', '__init_subclass__', '__int__', '__invert__', '__ior__', '__ipow__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__long__', '__lt__', '__matmul__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdiv__', '__rdiv

In [21]:

my_dict = {}
for deet in deets:
    for d in deet.split(','):
        if ":" in d:
            it = iter([(d.split(":")[0]).strip(), (d.split(":")[1]).strip()])
            mydict = dict(zip(it, it))
            my_dict.update(mydict)

        else:
            l = (d.strip()+":True").split(":")
            it = iter(l)
            myd = dict(zip(it, it))
            # print(myd)
            my_dict.update(myd)
print(my_dict)

{'Heating': 'Forced Air', 'Days on Market': '20 Days on Trulia', 'Year Built': '1963', 'Property Type': 'Apartment', 'Microwave': 'True', 'Air Conditioning': 'True', 'Hot Tub or Spa': 'True', 'Balcony': 'True', 'Parking': 'None', 'Pool': 'True', 'Cats': 'True', 'dogs allowed': 'True', 'Fitness Center': 'True', "['Heating": "Electric'", "'Days on Market": "36 Days on Trulia'", "'Year Built": "2008'", "'Property Type": "Apartment'", "'Microwave'": 'True', "'Air Conditioning'": 'True', "'Hot Tub or Spa'": 'True', "'Balcony'": 'True', "'Parking": "Off Street'", "'Pool'": 'True', "'Cats": 'True', "dogs allowed'": 'True', "'Fitness Center']": 'True', 'Additional Storage': 'True', 'Dishwasher': 'True', 'Disposal': 'True', 'Dryer': 'True', 'Laundry Facilities': 'True', 'Washer': 'True', 'Window Coverings': 'True', 'Disabled Access': 'True', 'Gated Entry': 'True', 'Patio': 'True', 'Assigned Parking Space': 'True', 'Ceiling Fan': 'True', 'Garage': 'True', "'Additional Storage'": 'True', "'Dishwa

In [22]:
it = iter((deets[0].split(".")[0]).split(":"))

In [23]:
print(next(it))

Heating


In [24]:
lets = "A B B N D I K H J R E".split(" ")
lets

['A', 'B', 'B', 'N', 'D', 'I', 'K', 'H', 'J', 'R', 'E']

In [25]:
it = iter(lets)

In [26]:
print(list(zip(it, it)))

[('A', 'B'), ('B', 'N'), ('D', 'I'), ('K', 'H'), ('J', 'R')]


In [40]:
my_dict = {}
deet =  deets[0]
d = deet.split(',')

if ":" in d[0]:
    # it = iter([(d[0].split(":")[0]).strip(), (d[0].split(":")[1]).strip()])
    # mydict = dict(zip(it, it))
    # my_dict.update(mydict)
    d1 = d[0].split(":")[0]
    d2 = d[0].split(":")[1]
    my_dict.update({d1: d2})
print(my_dict)


{'Heating': ' Other '}
